## Parameter setting and import libraries

In [1]:
from __future__ import print_function
#
import os
import glob
import re
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import time
from time import sleep
from tqdm import tqdm # if use notebook


from threading import Thread, Event, Timer
import queue

from PIL import Image
import cv2
import imgaug as ia
from imgaug import augmenters as iaa
import random

parser = argparse.ArgumentParser()
parser.add_argument('--gpu_id', default=5)
parser.add_argument('--image_dir', default="/data/seanyu/cat_dog/dataset/")
parser.add_argument('--save_dir', default='./result')
parser.add_argument('--batch_size', default=64, type=int)
parser.add_argument('--do_augment', default=True, type = bool)
parser.add_argument('--epochs', default=50, type=int)
parser.add_argument('--lr', default=0.0001, type=float)
parser.add_argument('--image_size', default=(256,256,3), type = int)
parser.add_argument('--n_classes', default=2, type = int)
parser.add_argument('--n_batch', default=100, type = int)
parser.add_argument('--train_ratio', default=0.99, type = float)
parser.add_argument('--use_model_ckpt', default = None, type = str)
parser.add_argument('--model_file_name', default = 'tmp_nb')
parser.add_argument('--n_threads', default = 4, type = int)
parser.add_argument('--dq_size', default = 6, type = int)

FLAGS = parser.parse_args([])
print(FLAGS)

/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  retu

Namespace(batch_size=64, do_augment=True, dq_size=6, epochs=50, gpu_id=5, image_dir='/data/seanyu/cat_dog/dataset/', image_size=(256, 256, 3), lr=0.0001, model_file_name='tmp_nb', n_batch=100, n_classes=2, n_threads=4, save_dir='./result', train_ratio=0.99, use_model_ckpt=None)


/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Check path and load data

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = str(FLAGS.gpu_id)
import tensorflow as tf


if not os.path.exists(FLAGS.save_dir):
    os.makedirs(FLAGS.save_dir)

model_dir = FLAGS.save_dir + '/model'
    
graphs_dir = FLAGS.save_dir + '/graphs'
if not os.path.exists(graphs_dir):
    os.makedirs(graphs_dir)

"""  Get data """
d_train = FLAGS.image_dir + '/train/'
d_test = FLAGS.image_dir + '/test1/'

image_train_list = glob.glob(d_train + '*.jpg')
image_test_list = glob.glob(d_test + '*.jpg')

df_train = pd.DataFrame({'img_path': image_train_list})
df_test = pd.DataFrame({'img_path': image_test_list})

df_train['cate'] = df_train.img_path.apply(os.path.basename)
df_train['cate'] = [i.split(".")[0] for i in list(df_train.cate)]
df_train.cate = df_train.cate.replace({'dog': 0, 'cat': 1})

nb_epoch = FLAGS.epochs

df_train_0, df_val_0 = train_test_split(df_train[df_train['cate'] == 0], test_size = 1-FLAGS.train_ratio)
df_train_1, df_val_1 = train_test_split(df_train[df_train['cate'] == 1], test_size = 1-FLAGS.train_ratio)

df_val = pd.concat((df_val_0, df_val_1)).reset_index(drop = True)

del df_val_0, df_val_1

/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
x_train = np.array([cv2.resize(cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB), (256,256)) for i in df_train.img_path], dtype=np.float32)
y_train = tf.keras.utils.to_categorical(df_train.cate.values, 2)

x_valid = np.array([cv2.resize(cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB), (256,256)) for i in df_val.img_path], dtype=np.float32)
y_valid = tf.keras.utils.to_categorical(df_val.cate.values, 2)

## Data Generator

In [14]:
"""
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    sometimes(iaa.Add((-10, 10), per_channel=0.5)),
    sometimes(iaa.ContrastNormalization((0.4, 0.6)))
])

def cv_load_and_resize(x, image_size, is_training = True, do_augment = False, seq = None):
    im_w, im_h, im_c = image_size
    im = cv2.imread(x)
    im = cv2.resize(im, (im_w, im_h))
    if do_augment and is_training:
        im = seq.augment_image(im)
    return im
"""

'\nsometimes = lambda aug: iaa.Sometimes(0.5, aug)\n\nseq = iaa.Sequential([\n    iaa.Fliplr(0.5),\n    iaa.Flipud(0.5),\n    sometimes(iaa.Add((-10, 10), per_channel=0.5)),\n    sometimes(iaa.ContrastNormalization((0.4, 0.6)))\n])\n\ndef cv_load_and_resize(x, image_size, is_training = True, do_augment = False, seq = None):\n    im_w, im_h, im_c = image_size\n    im = cv2.imread(x)\n    im = cv2.resize(im, (im_w, im_h))\n    if do_augment and is_training:\n        im = seq.augment_image(im)\n    return im\n'

## Build model

In [141]:
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as slimNet
from resnet import resnet_v2_50
import resnet_utils

tf.reset_default_graph()
tf.keras.backend.clear_session()
tf.keras.backend.set_learning_phase(True)

input_holder = tf.keras.layers.Input(shape=[256, 256, 3], name="inputs")

# Lambda layer form
def lambda_pretrain(x):
    with slim.arg_scope(resnet_utils.resnet_arg_scope(batch_norm_decay=0.95)):
        net, layers_dict = resnet_v2_50(inputs=x, is_training=True)
    
    return net

# Custom layer form
class Wrap_pretrain(tf.keras.layers.Layer):
    def __init__(self, input):
        super(Wrap_pretrain, self).__init__()
        self.tensor_input = input
        
    def build(self, input_shape):
        
        with slim.arg_scope(resnet_utils.resnet_arg_scope(batch_norm_decay=0.95)):
            self.net, layers_dict = resnet_v2_50(inputs=self.tensor_input, is_training=tf.keras.backend.learning_phase())
        
        """
        total_parameters = 0
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            #print(shape)
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        
        self.kernel = self.add_weight(name='pretrain_parameters',
                                       shape=(input_shape[1], total_parameters),
                                       initializer='uniform',
                                       trainable=True)
        print(self.kernel)
        """
        self.kernel = []
        for i, layer in enumerate(tf.trainable_variables()):
            if i == 0:
                w = self.add_weight(name='l'+str(i),
                                    shape=(input_shape[]))
                self.kernel.append()
        
        super(Wrap_pretrain, self).build(input_shape)
        
    
    def call(self, input):
        return self.net
    
    def compute_output_shape(self, input_shape):
        return tuple(input_shape)

#net = tf.keras.layers.Lambda(lambda_pretrain)(input_holder)
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters

net = Wrap_pretrain(input_holder)(input_holder)

gap_layer = tf.keras.layers.GlobalAveragePooling2D(name="GAP_Layer")(net)
out = tf.keras.layers.Dense(2, name='output', activation='softmax')(gap_layer)


optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

model = tf.keras.Model(inputs=[input_holder], outputs=[out])
model.compile(loss="categorical_crossentropy", 
              metrics=["accuracy"], 
              optimizer=optimizer)

for layer in model.layers:
    layer.trainable = True
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    update = optimizer.minimize(model.total_loss)

model.summary()


print("Numbers of Trainable parameters in Wrapped layer: %i " % total_parameters)

<tf.Variable 'wrap_pretrain/pretrain_parameters:0' shape=(256, 23519360) dtype=float32_ref>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 256, 256, 3)       0         
_________________________________________________________________
wrap_pretrain_1 (Wrap_pretra (None, 1, 1, 2048)        6020956160
_________________________________________________________________
GAP_Layer (GlobalAveragePool (None, 2048)              0         
_________________________________________________________________
output (Dense)               (None, 2)                 4098      
Total params: 6,020,960,258
Trainable params: 6,020,960,258
Non-trainable params: 0
_________________________________________________________________
Numbers of Trainable parameters in Wrapped layer: 0 


In [46]:
# Directly use tf.keras.application
tf.reset_default_graph()
tf.keras.backend.clear_session()

input_holder = tf.keras.layers.Input(shape=[256, 256, 3], name="inputs")
pretrain = tf.keras.applications.ResNet50(include_top=False, classes=2, weights=None)(input_holder)
gap_layer = tf.keras.layers.Flatten()(pretrain)
out = tf.keras.layers.Dense(2, activation='softmax')(gap_layer)

model = tf.keras.Model(inputs=[input_holder], outputs=[out])
model.compile(loss="categorical_crossentropy", 
              metrics=["accuracy"], 
              optimizer=tf.keras.optimizers.Adam(lr=0.001))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 256, 256, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


## Start training

In [139]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_valid, y_valid))

Train on 25000 samples, validate on 252 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[256,23519360] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: wrap_pretrain/pretrain_parameters/Initializer/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, _class=["loc:@wrap_pretrain/pretrain_parameters/Assign"], dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](wrap_pretrain/pretrain_parameters/Initializer/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'wrap_pretrain/pretrain_parameters/Initializer/random_uniform/RandomUniform', defined at:
  File "/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  F

4